In [4]:
import pandas as pd
import codecs
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime
#F=codecs.open('xxx.csv','r','utf-8')
#f= codecs.open('部分数据/附件1/YD1.xlsx','r', 'utf8')

In [5]:
path = u"C:/Users/Administrator/git/NILM/A题"
yd1 = pd.ExcelFile(path+ u"部分数据/附件1/YD1.xlsx")
yd2 = pd.ExcelFile(path+ u"部分数据/附件1/YD2.xlsx")
yd3 = pd.ExcelFile(path+ u"部分数据/附件1/YD3.xlsx")
yd4 = pd.ExcelFile(path+ u"部分数据/附件1/YD4.xlsx")
yd5 = pd.ExcelFile(path+ u"部分数据/附件1/YD5.xlsx")
yd6 = pd.ExcelFile(path+ u"部分数据/附件1/YD6.xlsx")
yd7 = pd.ExcelFile(path+ u"部分数据/附件1/YD7.xlsx")
yd8 = pd.ExcelFile(path+ u"部分数据/附件1/YD8.xlsx")
yd9 = pd.ExcelFile(path+ u"部分数据/附件1/YD9.xlsx")
yd10 = pd.ExcelFile(path+ u"部分数据/附件1/YD10.xlsx")
yd11 = pd.ExcelFile(path+ u"部分数据/附件1/YD11.xlsx")
yd3yd7 = pd.ExcelFile(path+ u"部分数据/附件3/设备组1-YD3+YD7.xlsx")
yd1yd2yd4 = pd.ExcelFile(path+ u"部分数据/附件3/设备组2-YD1+YD2+YD4.xlsx")
yd4yd5yd10yd11=pd.ExcelFile(path+ u"部分数据/附件3/设备组3-YD4+YD5+YD8+YD10+YD11.xlsx")
yd=[yd1,yd2,yd3,yd4,yd5,yd6,yd7,yd8,yd9,yd10, yd11, yd3yd7, yd1yd2yd4, yd4yd5yd10yd11]

equip_data = []
period_data = []
harmonic_data = []
oper_data = []
for i in range(0,14):
    equip_data.append(yd[i].parse(u'设备数据'));
    period_data.append(yd[i].parse(u'周波数据'));
    harmonic_data.append(yd[i].parse(u'谐波数据'));
for i in range(0,11):
    oper_data.append(yd[i].parse(u'操作记录'));
oper_data[1].shape

(9, 5)

### Define two functions to deal with missing operation time in equip_data

In [6]:
def find_next(datetimes, datetime_point):
    '''
    datetimes : An array of numpy.datetime.
    datetime_point: An datetime that need to be found.
    return : the first available datetime and its index in datetimes array that is bigger than datetime_point.
             If the datetime_point is bigger than all the values in datetimes, return None
    '''
    for i in range(len(datetimes)):
        if datetimes[i]<datetime_point:
            found = False
            continue
        if datetimes[i]>datetime_point:
            return i, datetimes[i]
    if found == False:
        return None
def find_prior(datetimes, datetime_point)  :
    '''
    datetimes : An array of numpy.datetime.
    datetime_point: An datetime that need to be found.
    return : the last available datetime and its index in datetimes array that is no bigger than datetime_point.
             If the datetime_point is smaller than all the values in datetimes, return None
    '''
    if datetimes[0]>datetime_point:
        return None
    
    for i in range(len(datetimes)):
        if datetimes[i]<datetime_point:
            continue
        if datetimes[i]>datetime_point:
            return i-1, datetimes[i-1]
    return i, datetimes[i]

### Define a function to get the  p,q value of steady states

In [7]:
def get_steady(i):
    '''
    input i: the equipment number. For example, for yd1, i=1
    return : the steady state name and the corresponding p , q value
    '''
    i=i-1
    data_p=[]
    data_q=[]
    label=[]
    p = equip_data[i]['P'].values
    q = equip_data[i]['Q'].values
    for ii in range(oper_data[i].shape[0]-1):
        index_1 = np.where(equip_data[i]['time'].values == oper_data[i][u'时间'].values[ii])[0]
        index_2 = np.where(equip_data[i]['time'].values == oper_data[i][u'时间'].values[ii+1])[0]
        if len(index_2)==0:
            index_2 = find_next(equip_data[i]['time'].values, oper_data[i][u'时间'].values[ii+1])
            if index_2 == None: # if index_2 is bigger than all the availabel datetimes, then return the find_prior
                index_2 = find_prior(equip_data[i]['time'].values, oper_data[i][u'时间'].values[ii+1])[0]+2
            else:
                index_2 = index_2[0] + 2
        if len(index_1)==0:
            index_1 = find_next(equip_data[i]['time'].values, oper_data[i][u'时间'].values[ii])
            if index_1 == None: # if index_1 is bigger than all the available datetimes, then there is no need to search
                break
            else:
                index_1 = index_1[0] - 2
        index_1 = int(index_1)
        index_2 = int(index_2)
        label.append('yd{}'.format(i+1) + ':' + oper_data[i][u'工作状态'].values[ii])
        data_p.append(np.mean(p[index_1+2:index_2-2]))
        data_q.append(np.mean(q[index_1+2:index_2-2]))
    return label, data_p, data_q

### Create a dataframe to store the steady state data. For microwave, laptop, and printer, because their steady state is hard to define, we did not include them.

In [8]:
steady_label = []
steady_p = []
steady_q = []
for i in [1,3,5,6,8,9,10,11]:
    label, data_p, data_q = get_steady(i)
    steady_label = steady_label + label
    steady_p = steady_p + data_p
    steady_q = steady_q + data_q
steady_dict = {
    'Steady State': steady_label,
    'P value' : steady_p,
    'Q value' : steady_q
    }
steady_df = pd.DataFrame(steady_dict)

In [9]:
steady_df

,P value,Q value,Steady State
0,2.944444,0.018519,yd1:关闭
1,285.275862,146.965517,yd1:1档
2,334.500000,132.250000,yd1:2档
3,381.125000,80.928571,yd1:3档
4,2.979592,0.040816,yd3:关闭
5,17043.676056,20.183099,yd3:打开
6,2.843137,0.039216,yd5:关闭
7,407.344262,6.180328,yd5:打开
8,2.816327,0.000000,yd6:关闭
9,53.100000,11.100000,yd6:打开
